In [ ]:
# Import packages and set configuration
import pandas as pd
from IPython.display import display

%load_ext autoreload
%autoreload 2

### Asset list
Market Cap Ranking Comparison Based on Historical CoinMarketCap Snapshots.

In [ ]:
# Load tickers
tickers = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "ADAUSDT", "AVAXUSDT", "DOGEUSDT", "TRXUSDT",
           "DOTUSDT", "LINKUSDT", "SHIBUSDT", "LTCUSDT", "BCHUSDT", "UNIUSDT"]
pd.read_excel('../data/asset_list.xlsm', dtype={'07.01.2024': 'Int64', '05.10.2025': 'Int64'}).iloc[:15, :2]

### Benchmark

#### Basic pair selection
- ##### Pair Selection Range: 01.01.2024 - 01.03.2024
- ##### Interval: 1h

In [ ]:
# Load data
from modules.pair_selection.statistical_tests import perform_statistical_tests
from modules.data_services.data_pipeline import load_data

ps_start = "2024-01-01"
ps_end = "2024-03-01"
interval = "1h"

df = load_data(
    tickers=tickers,
    start=ps_start,
    end=ps_end,
    interval=interval
)

In [ ]:
# Statistical tests
corr_coint_tests = perform_statistical_tests(df)
display(corr_coint_tests)

#### Global parameters
- ##### Trading Range: 01.03.2024 - 01.04.2024
- ##### Interval: 1h
- ##### Fee Rate: 0.05% per Position
- ##### Position Size: Static, Always 100% (50% Long, 50% Short)

In [ ]:
# Set parameters
trading_start = "2024-03-01"
trading_end = "2024-04-01"
fee_rate = 0.0005  # 0.05%
position_size = 1  # always 100% of portfolio

#### 1. Benchmark (Gatev et al. 2006)
- ##### Pair Selection Method: Top 5 of SSD of Cumulative Returns Test
- ##### Z-Score: Calculated from Cumulative Returns
- ##### Entry Threshold: Static
- ##### Exit Threshold: Static

In [ ]:
# Pair Selection
from modules.pair_selection.statistical_tests import sum_of_standard_deviation

ssd_df = sum_of_standard_deviation(df)
pairs_1 = ssd_df.iloc[0:5, 0].tolist()
display(ssd_df.head(5))

In [17]:
# Find the best rolling window and thresholds
from skopt.space import Real, Integer
from skopt.utils import use_named_args

from modules.data_services.param_optimization import bayesian_optimization
from modules.performance.strategy import run_strategy


param_space = [
    Integer(1, 30*24, name='window_in_steps'),
    Real(0.0, 5.0, name='entry_threshold'),
    Real(0.0, 3.0, name='exit_threshold'),
]

# Objective function of Bayesian Optimization
@use_named_args(param_space)
def objective(**params):
    p = run_strategy(
        pairs=pairs_1,
        trading_start=trading_start,
        trading_end=trading_end,
        interval=interval,
        position_size = position_size,
        z_score_method='cum_returns',
        fee_rate=fee_rate,
        window_in_steps=params['window_in_steps'],
        entry_threshold=params['entry_threshold'],
        exit_threshold=params['exit_threshold']
    )
    return p.summary['sharpe_ratio']['0.05% fee']['Summary']

best_params_1, best_score_1, res_1 = bayesian_optimization(param_space=param_space, objective=objective, n_calls=40, random_state=42)
print(best_params_1)
print(best_score_1)

{'window_in_steps': np.int64(27), 'entry_threshold': 4.087615572445448, 'exit_threshold': 2.96287952002781}
5.834121598435865


In [18]:
# Perform strategy
from modules.visualization.plots import plot_summary_pnl


window_in_steps = best_params_1['window_in_steps']
entry_threshold = best_params_1['entry_threshold']
exit_threshold = best_params_1['exit_threshold']

portfolio_1 = run_strategy(
    pairs=pairs_1,
    trading_start=trading_start,
    trading_end=trading_end,
    interval=interval,
    position_size = position_size,
    z_score_method='cum_returns',
    fee_rate=fee_rate,
    window_in_steps=window_in_steps,
    entry_threshold=entry_threshold,
    exit_threshold=exit_threshold
)
plot_summary_pnl(portfolio_1, '1')
display(portfolio_1.summary)

total_pnl                total_return            \
                          0% fee      0.05% fee       0% fee 0.05% fee   
DOGEUSDT-SHIBUSDT   57718.641662   53218.641662     5.771864  5.321864   
XRPUSDT-DOTUSDT     -7736.363084   -9736.363084    -0.773636 -0.973636   
ADAUSDT-DOTUSDT     31973.351410   29473.351410     3.197335  2.947335   
SHIBUSDT-LTCUSDT    84213.351114   80713.351114     8.421335  8.071335   
ADAUSDT-AVAXUSDT    -4276.094568   -8276.094568    -0.427609 -0.827609   
Summary            161892.886535  145392.886535     3.237858  2.907858   

                  annualized_volatility           sharpe_ratio            \
                                 0% fee 0.05% fee       0% fee 0.05% fee   
DOGEUSDT-SHIBUSDT              0.091540  0.091884     6.817387  6.279575   
XRPUSDT-DOTUSDT                0.085764  0.085947    -0.957460 -1.214559   
ADAUSDT-DOTUSDT                0.064560  0.064769     5.417378  4.986421   
SHIBUSDT-LTCUSDT               0.139003  0.139303     6.494299  6.224165   
ADAUSDT-AVAXUSDT               0.145275  0.145707    -0.253617 -0.557606   
Summary                        0.054465  0.054529     6.490272  5.834122   

                  max_drawdown            
                        0% fee 0.05% fee  
DOGEUSDT-SHIBUSDT    -0.753742 -0.803560  
XRPUSDT-DOTUSDT      -2.566278 -2.667055  
ADAUSDT-DOTUSDT      -0.614692 -0.664673  
SHIBUSDT-LTCUSDT     -1.113695 -1.162690  
ADAUSDT-AVAXUSDT     -4.144981 -4.250752  
Summary              -1.026117 -1.056926

#### 2. Benchmark (Yang & Malik, 2024)
- ##### Pair Selection Method: Top 5 of Correlation of Log Returns and Engle-Granger Cointegration Test (Yang & Malik 2024)
- ##### Z-Score: calculated from cumulative returns
- ##### Entry Threshold: static
- ##### Exit Threshold: static

In [15]:
# Pair Selection
from modules.data_services.data_pipeline import merge_by_pair
from modules.pair_selection.statistical_tests import pearson_correlation, engle_granger_cointegration

corr_df = pearson_correlation(df, source='log_returns')
eg_df = engle_granger_cointegration(df)
merged_df = merge_by_pair(
    dfs=[corr_df, eg_df],
    keep_cols=[
        ['corr_log_returns'],
        ['eg_p_value']
    ]
)
merged_df['corr_log_returns * (1 - eg_p_value)'] = merged_df['corr_log_returns'] * (1 - merged_df['eg_p_value'])
merged_df.sort_values(by=['corr_log_returns * (1 - eg_p_value)'], ascending=False, inplace=True)
merged_df.reset_index(drop=True, inplace=True)
pairs_2 = merged_df.iloc[0:5, 0].tolist()
display(merged_df.head(5))

,pair,corr_log_returns,eg_p_value,corr_log_returns * (1 - eg_p_value)
0,ADAUSDT-DOTUSDT,0.826138,0.132649,0.716552
1,ETHUSDT-ADAUSDT,0.753668,0.064789,0.704838
2,BTCUSDT-ADAUSDT,0.711226,0.013955,0.701301
3,ETHUSDT-DOTUSDT,0.740521,0.100618,0.666012
4,BTCUSDT-SOLUSDT,0.674951,0.015325,0.664608


In [16]:
# Find the best rolling window and thresholds
from skopt.space import Real, Integer
from skopt.utils import use_named_args

from modules.data_services.param_optimization import bayesian_optimization
from modules.performance.strategy import run_strategy


param_space = [
    Integer(1, 30*24, name='window_in_steps'),
    Real(0.0, 5.0, name='entry_threshold'),
    Real(0.0, 3.0, name='exit_threshold'),
]

# Objective function of Bayesian Optimization
@use_named_args(param_space)
def objective(**params):
    p = run_strategy(
        pairs=pairs_2,
        trading_start=trading_start,
        trading_end=trading_end,
        interval=interval,
        position_size = position_size,
        z_score_method='rolling_beta',
        fee_rate=fee_rate,
        window_in_steps=params['window_in_steps'],
        entry_threshold=params['entry_threshold'],
        exit_threshold=params['exit_threshold']
    )
    return p.summary['sharpe_ratio']['0.05% fee']['Summary']

best_params_2, best_score_2, res_2 = bayesian_optimization(param_space=param_space, objective=objective, n_calls=40, random_state=42)
print(best_params_2)
print(best_score_2)

{'window_in_steps': np.int64(66), 'entry_threshold': 2.4718124128647725, 'exit_threshold': 0.9751080067380108}
5.435181782059752


In [20]:
# Perform strategy
from modules.visualization.plots import plot_summary_pnl


window_in_steps = best_params_2['window_in_steps']
entry_threshold = best_params_2['entry_threshold']
exit_threshold = best_params_2['exit_threshold']

portfolio_2 = run_strategy(
    pairs=pairs_2,
    trading_start=trading_start,
    trading_end=trading_end,
    interval=interval,
    position_size=position_size,
    z_score_method='rolling_beta',
    fee_rate=fee_rate,
    window_in_steps=window_in_steps,
    entry_threshold=entry_threshold,
    exit_threshold=exit_threshold
)
plot_summary_pnl(portfolio_2, '2')
display(portfolio_2.summary)

total_pnl                total_return             \
                        0% fee      0.05% fee       0% fee  0.05% fee   
ADAUSDT-DOTUSDT  141291.179085  132791.179085    14.129118  13.279118   
ETHUSDT-ADAUSDT   50673.532678   41673.532678     5.067353   4.167353   
BTCUSDT-ADAUSDT   69528.333656   62028.333656     6.952833   6.202833   
ETHUSDT-DOTUSDT   78051.320794   69051.320794     7.805132   6.905132   
BTCUSDT-SOLUSDT  -20674.487025  -27174.487025    -2.067449  -2.717449   
Summary          318869.879188  278369.879188     6.377398   5.567398   

                annualized_volatility           sharpe_ratio            \
                               0% fee 0.05% fee       0% fee 0.05% fee   
ADAUSDT-DOTUSDT              0.154138  0.154743     9.270566  8.716877   
ETHUSDT-ADAUSDT              0.184972  0.185711     2.957828  2.450067   
BTCUSDT-ADAUSDT              0.182629  0.183140     4.037392  3.614605   
ETHUSDT-DOTUSDT              0.238324  0.239150     3.500468  3.113295   
BTCUSDT-SOLUSDT              0.221892  0.222593    -0.898553 -1.215790   
Summary                      0.117823  0.118300     5.684996  4.969765   

                max_drawdown            
                      0% fee 0.05% fee  
ADAUSDT-DOTUSDT    -2.130155 -2.131218  
ETHUSDT-ADAUSDT    -6.933218 -6.991659  
BTCUSDT-ADAUSDT    -5.017041 -5.021831  
ETHUSDT-DOTUSDT    -3.877186 -3.978860  
BTCUSDT-SOLUSDT    -5.303637 -5.463319  
Summary            -2.523541 -2.545967